In [1]:
import os
import numpy as np
import pandas as pd
import csv

print(os.getcwd())
CARPOL_1303_DATAFOLDER = 'data/carpol_1303'
CARPOL_1304_DATAFOLDER = 'data/carpol_1304'
GLOBAL_DATAFOLDER = 'data'

labels = {
    'date': 'date',
    'H2S(ppb)': 'h2s',
    'Temperature(°C)': 'temperature',
    'RelativeHumidity(%)': 'humidity',
    'Patm(hPa)': 'pressure',
    'SO2(ppb)': 'so2'
}

/Users/alexis/Documents/air_normand_pao


In [3]:
# Aperçu des csv
test_df = pd.read_csv(os.path.join(CARPOL_1303_DATAFOLDER, '1303_15minDataSet.txt'), encoding = "ISO-8859-1")
test_df.head()

,# CairPol-1303 données 15min
0,# Time GMT
1,# date;H2S(ppb);Temperature(°C);RelativeHumidi...
2,2016-05-23 08:15:00;7.6;24;42;8
3,2016-05-23 08:30:00;14.1;24;42;15
4,2016-05-23 08:45:00;14.9;24;42;15


In [4]:
# On transforme les txt en .csv pour une quesstion pratique

def txt_data_to_csv(filename):
    csvfile = open(".".join([filename.split('.')[0], 'csv']), 'w', newline='')
    writer = csv.writer(csvfile, delimiter=',' , quotechar='|', quoting=csv.QUOTE_MINIMAL)
    with open(filename, 'r', encoding = "ISO-8859-1") as f:
        lines = f.readlines()
    i = 0
    while lines[i][0] == "#":
        i += 1
    header_row = [param.strip() for param in lines[i-1].split("#")[1].split(";")]
    writer.writerow(header_row)
    for line in lines[i:]:
        writer.writerow([val.strip() for val in line.split(";")])
        
txt_data_to_csv(os.path.join(CARPOL_1303_DATAFOLDER, '1303_15minDataSet.txt'))
txt_data_to_csv(os.path.join(CARPOL_1303_DATAFOLDER, '1303_1minDataSet.txt'))
txt_data_to_csv(os.path.join(CARPOL_1304_DATAFOLDER, '1304_15minDataSet.txt'))
txt_data_to_csv(os.path.join(CARPOL_1304_DATAFOLDER, '1304_1minDataSet.txt'))
txt_data_to_csv(os.path.join(GLOBAL_DATAFOLDER, 'Patm_15minDataSet.txt'))
txt_data_to_csv(os.path.join(GLOBAL_DATAFOLDER, 'RefInstrument_15minDataSet.txt'))

In [99]:
# Création du pickle (Un peu long donc ne pas lancer à chaque fois) -> Pandas très lent mais utile pour générer le pickle

# Inversion des dataframes de données pour accéder aux infos avec la date comme clé
df = pd.read_csv(os.path.join(GLOBAL_DATAFOLDER, 'Patm_15minDataSet.csv'))
pressures = df.set_index('date').T  # idx 0 -> Patm

df = pd.read_csv(os.path.join(GLOBAL_DATAFOLDER, 'RefInstrument_15minDataSet.csv'))
references = df.set_index('date').T # idx 0 -> H2S, idx 1 -> SO2
print(references['2016-11-06 15:30:00'])
print(references['2016-11-06 15:30:00'][1])
#print(references['2016-05-23 00:15:00'], pressures['2016-05-23 00:15:00'])
#print(references.head())
# Création du pickle
captors_list_filename = ((os.path.join(CARPOL_1303_DATAFOLDER, '1303_15minDataSet.csv'), 1303), (os.path.join(CARPOL_1304_DATAFOLDER, '1304_15minDataSet.csv'), 1304))

out_df = pd.DataFrame(columns=['date','h2s','pressure', 'temperature','humidity','n_points','so2','h2s_ref', 'captor'])

#entry[0]date, entry[1] h2s, entry[2] temp, entry[3] humidity, entry[4] npoints
i = 0

for filename in captors_list_filename:
    df = pd.read_csv(filename[0])
    #print(df.head())
    for row in df.iterrows():
        entry = row[1]
        try:
            date = entry['date']
        except:
            continue
        #print(i)
        #print(filename)
        try:
            ref = references[date]
            out_df.loc[i] = [date, entry[1], pressures[date][0], entry[2], entry[3], entry[4], ref[1], ref[0], filename[1]]
        except KeyError:
            out_df.loc[i] = [date, entry[1], pressures[date][0], entry[2], entry[3], entry[4], "N\A", "N\A", filename[1]]
        i += 1
        
print(out_df.head())
out_df.to_pickle('data/data.pkl')

H2S(ppb)    0.7
SO2(ppb)    0.4
Name: 2016-11-06 15:30:00, dtype: float64
0.4
                  date   h2s pressure  temperature  humidity n_points so2  \
0  2016-05-23 08:15:00   7.6     1017         24.0      42.0        8   0   
1  2016-05-23 08:30:00  14.1     1017         24.0      42.0       15   0   
2  2016-05-23 08:45:00  14.9     1017         24.0      42.0       15   0   
3  2016-05-23 09:00:00  15.0     1017         24.0      42.0       15   0   
4  2016-05-23 09:15:00  14.9     1017         24.0      42.0       15   0   

  h2s_ref captor  
0     NaN   1303  
1     NaN   1303  
2     NaN   1303  
3     NaN   1303  
4       0   1303  


In [79]:
"""
# On centre et réduit et on save dans chaque csv
def normalize_carpol_csv(filename):
    df = pd.read_csv(filename)
    df.iloc[:, 1:] = test.iloc[:, 1:-1].apply(lambda x: (x - np.mean(x)) / (np.std(x)))
    filename_split = filename.split('.')
    df.to_csv("{}{}.{}".format(filename_split[0], '_norm', filename_split[1]), index=False)
    
normalize_carpol_csv(os.path.join(CARPOL_1303_DATAFOLDER, '1303_15minDataSet.csv'))
normalize_carpol_csv(os.path.join(CARPOL_1303_DATAFOLDER, '1303_1minDataSet.csv'))
normalize_carpol_csv(os.path.join(CARPOL_1304_DATAFOLDER, '1304_15minDataSet.csv'))
normalize_carpol_csv(os.path.join(CARPOL_1304_DATAFOLDER, '1304_1minDataSet.csv'))
"""

In [158]:
# Essai read pickle
df = pd.read_pickle('data/data.pkl')
data = df[['h2s', 'pressure', 'temperature', 'humidity', 'so2']]
df[['h2s', 'pressure', 'temperature', 'humidity', 'so2']] =  data.apply(pd.to_numeric, errors="coerce").apply(lambda x: (x - np.mean(x)) / (np.std(x)))
print(len(df))
df = df[pd.notnull(df).all(axis=1)] # On ne garde que les données sans NaN etc..
print(len(df))
df.head()

19445
8676


,date,h2s,pressure,temperature,humidity,n_points,so2,h2s_ref,captor
4,2016-05-23 09:15:00,1.673433,0.047316,0.996046,-1.610853,15,-0.259294,0,1303
9,2016-05-23 10:30:00,1.530588,0.202130,0.996046,-1.677224,15,-0.259294,0,1303
20,2016-05-23 13:15:00,1.637722,0.356944,0.996046,-1.809965,15,-0.259294,0,1303
25,2016-05-23 14:30:00,1.655577,0.356944,0.996046,-1.809965,15,-0.259294,0.1,1303
31,2016-05-23 16:00:00,1.566299,0.356944,0.996046,-1.876336,15,-0.259294,0,1303


In [2]:
df = pd.read_pickle('data/data.pkl')
data = df[['h2s', 'pressure', 'temperature', 'humidity', 'so2', 'h2s_ref']]
df[['h2s', 'pressure', 'temperature', 'humidity', 'so2', 'h2s_ref']] =  data.apply(pd.to_numeric, errors="coerce").apply(lambda x: (x - np.mean(x)) / (np.std(x)))


In [7]:
df.to_pickle('data/data_15min.pkl')